In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [4]:
df=pd.read_csv("sunspots.csv")
df["Month"]=pd.to_datetime(df["Month"])
df.set_index('Month', inplace=True)
df

,Sunspots
Month,
1749-01-01,58.0
1749-02-01,62.6
1749-03-01,70.0
1749-04-01,55.7
1749-05-01,85.0
...,...
1983-08-01,71.8
1983-09-01,50.3
1983-10-01,55.8


In [5]:
df.dtypes

Sunspots    float64
dtype: object

In [6]:
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size], df.iloc[train_size:]

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_date = train.index
test_date = test.index

In [8]:
train_values = train.values.reshape(-1, 1)
test_values = test.values.reshape(-1, 1)

In [9]:
train_scaled = scaler.fit_transform(train_values)
test_scaled = scaler.transform(test_values)

In [10]:
train_scaled = train_scaled.reshape(-1, len(train.columns))
test_scaled = test_scaled.reshape(-1, len(test.columns))

In [11]:
train_scaled_df = pd.DataFrame(train_scaled, columns=train.columns, index=train_date)
test_scaled_df = pd.DataFrame(test_scaled, columns=test.columns, index=test_date)

In [12]:
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length + 1):
        seq = data[i:i + sequence_length]
        sequences.append(seq)
    return np.array(sequences)

In [13]:
sequence_length = 10
X_train = create_sequences(train_scaled_df.values, sequence_length)
X_test = create_sequences(test_scaled_df.values, sequence_length)

In [14]:
X_train, y_train = X_train[:, :-1], X_train[:, -1][:, -1]
X_test, y_test = X_test[:, :-1], X_test[:, -1][:, -1]

In [15]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [16]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=2)

Epoch 1/50
71/71 - 2s - loss: 0.0176 - val_loss: 0.0107 - 2s/epoch - 25ms/step
Epoch 2/50
71/71 - 0s - loss: 0.0050 - val_loss: 0.0085 - 252ms/epoch - 4ms/step
Epoch 3/50
71/71 - 0s - loss: 0.0048 - val_loss: 0.0082 - 225ms/epoch - 3ms/step
Epoch 4/50
71/71 - 0s - loss: 0.0047 - val_loss: 0.0080 - 242ms/epoch - 3ms/step
Epoch 5/50
71/71 - 0s - loss: 0.0047 - val_loss: 0.0077 - 238ms/epoch - 3ms/step
Epoch 6/50
71/71 - 0s - loss: 0.0045 - val_loss: 0.0082 - 266ms/epoch - 4ms/step
Epoch 7/50
71/71 - 0s - loss: 0.0045 - val_loss: 0.0073 - 256ms/epoch - 4ms/step
Epoch 8/50
71/71 - 0s - loss: 0.0044 - val_loss: 0.0072 - 232ms/epoch - 3ms/step
Epoch 9/50
71/71 - 0s - loss: 0.0044 - val_loss: 0.0072 - 246ms/epoch - 3ms/step
Epoch 10/50
71/71 - 0s - loss: 0.0043 - val_loss: 0.0073 - 248ms/epoch - 3ms/step
Epoch 11/50
71/71 - 0s - loss: 0.0044 - val_loss: 0.0071 - 243ms/epoch - 3ms/step
Epoch 12/50
71/71 - 0s - loss: 0.0043 - val_loss: 0.0069 - 232ms/epoch - 3ms/step
Epoch 13/50
71/71 - 0s - lo

In [17]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

18/18 [==============================] - 0s 2ms/step
Mean Squared Error: 0.006354680440828007
Mean Absolute Error: 0.05818939481380853
